In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore
import boto3
import os
from azure.storage.blob import BlobServiceClient
from tqdm import tqdm
import json
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.retrievers import BM25Retriever
from langchain.prompts import ChatPromptTemplate
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from operator import itemgetter
from langchain.retrievers import EnsembleRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnableParallel
import numpy as np
from dotenv  import load_dotenv
load_dotenv()
import shutil

def load_file(file_name):
    loader=[]
    print(file_name.split(".")[-1])
    if file_name.split('.')[-1] == "pptx":
        loader = UnstructuredPowerPointLoader(file_name).load()
    elif file_name.split('.')[-1] == "pdf":
        loader = PyPDFLoader(file_name).load()    
    elif file_name.split('.')[-1] == "docx":
        loader = Docx2txtLoader(file_name).load()
    elif file_name.split('.')[-1] == "html":
        loader = UnstructuredHTMLLoader(file_name).load()
    
    text_splitter = RecursiveCharacterTextSplitter(
        # separator="\n\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False
        )
    pages = text_splitter.split_documents(loader)
    return pages


def file_to_chunks(folder):
    pages=[]
    for file_name in os.listdir(f"{folder}"):
        pages.extend(load_file(f"{folder}\\{file_name}"))
    if folder != "Local_data":
        shutil.rmtree(f"{folder}")
    return pages

def azure_data_download(AZURE_CONNECTION_STRING,CONTAINER_NAME):
    blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(CONTAINER_NAME)
    if not os.path.exists("Azure_data"):
        os.mkdir("Azure_data")
    for file_name in container_client.list_blobs():
        blob_client = container_client.get_blob_client(file_name)
        with open(f"Azure_data\\{file_name.name}", "wb") as file:
            data = blob_client.download_blob().readall()
            file.write(data)


def aws(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, BUCKET_NAME,object_name):
        # Create an S3 client
    s3 = boto3.client('s3',
                    aws_access_key_id=AWS_ACCESS_KEY_ID,
                    aws_secret_access_key=AWS_SECRET_ACCESS_KEY)

    # List objects in the bucket
    response = s3.list_objects_v2(Bucket=BUCKET_NAME)

    if not os.path.exists("S3_data"):
        os.mkdir("S3_data")

    # Download files in the 'data' object
    for i in response.get('Contents',[]):
        if i['Key'].split('/')[-1] != "" and i['Key'].split('/')[0] == object_name:
            # print(i['Key'])
            file_path = os.path.join("S3_data", i['Key'].split('/')[-1])
            # print(file_path)
            s3.download_file(BUCKET_NAME, i['Key'], file_path)


def l(h):
    d=""
    for i in h:
        for _,j in i.items():
            d=d+f"{j} \n"
    if d == "":
        d = "No history found"
    return d


def generate_queries(query):
        
    # Multi Query: Different Perspectives
    with open("history.json", "r") as f:
        h=json.load(f)
    h_1 =l(h)
    # print(h_1)
    # print(h)
    # print(type(h))
    template = f"""You are an AI language model assistant. Your task is to generate four different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
        Additionally, look for the relevant information from the history to provide more contextually accurate variations. Provide these alternative questions separated by newlines.
        
        history : {h_1}
        Original question: {query}
        """
    # print(template)
    # print(h_1)
    prompt_perspectives = ChatPromptTemplate.from_template(template)
    generate_querie = (
        prompt_perspectives
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
        | (lambda x: [query] + x)
    )
    return generate_querie 

def keyword_extractor():
    prompt="""
    You are an AI language model assistant. Your task is to help the user identify key terms in their query.

    Please list the main keywords you want to extract from your query.

    query: {query}
    """
    prompt_perspectives=ChatPromptTemplate.from_template(prompt)
    generate_querie = (
        prompt_perspectives 
        | ChatOpenAI(temperature=0) 
        | StrOutputParser() )
    return generate_querie

def _get(a):
    dd=[]
    for s in a:
        dd.extend(s)
    return dd

def get_unique_documents(doc_list):
    seen_content = set()
    unique_documents = []
    
    for doc in doc_list:
        content = doc.page_content
        if content not in seen_content:
            seen_content.add(content)
            unique_documents.append(doc)
    
    del seen_content
    
    return unique_documents

In [21]:
with open("history.json", "r") as f:
    h=json.load(f)
h_1 =l(h)
# print(h_1)
# print(h)
# print(type(h))
template = f"""You are an AI language model assistant. Your task is to generate four different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Additionally, look for the relevant information from the history to provide more contextually accurate variations. Provide these alternative questions separated by newlines.

history : """
template=template+h_1
template=template+"""
Original question: {query}
"""

print(template)
# print(h_1)
prompt_perspectives = ChatPromptTemplate.from_template(template,input_variables=["query","history"])

You are an AI language model assistant. Your task is to generate four different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
Additionally, look for the relevant information from the history to provide more contextually accurate variations. Provide these alternative questions separated by newlines.

history : No history found
Original question: {query}



TypeError: langchain_core.prompts.prompt.PromptTemplate() got multiple values for keyword argument 'input_variables'

In [17]:
prompt_perspectives

ChatPromptTemplate(input_variables=[], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an AI language model assistant. Your task is to generate four different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.\nAdditionally, look for the relevant information from the history to provide more contextually accurate variations. Provide these alternative questions separated by newlines.\n\nhistory : No history found\nOriginal question: What are the health benefits of green tea?\n'))])

In [9]:
generate_queries(Query).invoke(Query)

Human: You are an AI language model assistant. Your task is to generate four different versions of the given user question to retrieve relevant documents from a vector database. By generating multiple perspectives on the user question, your goal is to help the user overcome some of the limitations of the distance-based similarity search.
        Additionally, look for the relevant information from the history to provide more contextually accurate variations. Provide these alternative questions separated by newlines.
        
        history : No history found
        Original question: tell me more about the shyam sundar's project
        


TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>